In [1]:
# import block
from tools.decompositions import qr_factorize as factorizer
from tools.decompositions import validate
import numpy as np

In [2]:
# random matrix generation

# set the seed for reusability
np.random.seed(1)

# generate a random 100x100 matrix
A = np.random.rand(500, 500)
print('A=\n', A)

# check if the matrix is non singular
det = np.linalg.det(A)
print(f'\nNon singular, det = {det}' if det != 0 else 'Singular')

A=
 [[4.17022005e-01 7.20324493e-01 1.14374817e-04 ... 8.44329949e-01
  9.20206514e-01 2.27900290e-01]
 [8.74822096e-02 2.27309736e-01 3.14376616e-01 ... 6.68796606e-01
  3.25967207e-01 7.74477266e-01]
 [3.25809967e-01 8.89827341e-01 7.51707721e-01 ... 6.44941673e-01
  4.84315773e-01 9.67695246e-01]
 ...
 [1.51486251e-01 6.38941486e-01 3.63883195e-01 ... 5.85179335e-01
  1.02424223e-01 2.81061430e-01]
 [3.93525026e-01 5.39786668e-01 5.84946564e-01 ... 1.92432947e-01
  6.45890776e-02 6.96983660e-01]
 [1.98653619e-01 2.90040575e-01 3.86837686e-01 ... 3.11285210e-01
  9.74478628e-01 3.19562678e-01]]

Non singular, det = 2.699021353261159e+299


In [3]:
# compare the results for
from timeit import default_timer as timer

t1 = timer()
factorizer(A)
t1 = timer() - t1

t2 = timer()
np.linalg.qr(A)
t2 = timer() - t2

print(
    f'As we have already expected, the builtin NumPy decomposition function is ~{t1/t2:.2f}x faster.')

As we have already expected, the builtin NumPy decomposition function is ~40.76x faster.


In [4]:
# error analysis

Q1, R1 = factorizer(A)  # our function
Q2, R2 = np.linalg.qr(A)  # NumPy function

custom_err = validate(A, Q1, R1)
NumPy_err = validate(A, Q2, R2)
print(
    f'Custom QR decomposition function:\n\tError Threshold Satisfied: {str(custom_err[0]):<10}Maximum Absolute Error: {custom_err[1]}')
print(
    f'NumPy QR decomposition function:\n\tError Threshold Satisfied: {str(NumPy_err[0]):<10}Maximum Absolute Error: {NumPy_err[1]}')

Custom QR decomposition function:
	Error Threshold Satisfied: False     Maximum Absolute Error: 1.067949394917278e-06
NumPy QR decomposition function:
	Error Threshold Satisfied: True      Maximum Absolute Error: 6.439293542825908e-15


## Trying out a singular matrix

Our function is not able to decompose singular matrices. It also decomposes $A_{m\times n}$ into $Q_{m\times n}\times R_{n\times n}$ while the built-in function of NumPy (and SciPy) uses $A_{m\times n} = Q_{m\times m}\times R_{m\times n}$. Moreover, these functions employ algorithms that turn singular matrices into non singular ones by changing the sign of the diagonal matrix elements. Consider the following examle:

In [5]:
# comparison of signs in algorithms

M = np.array([
    [1, 1, 0],
    [1,  0, 1],
    [0,  1, 1]
])
Q_custom, R_custom = factorizer(M)
Q_NumPy, R_NumPy = np.linalg.qr(M)

print(f'Our Q:\n{str(Q_custom)}')
print(f'NumPy Q:\n{str(Q_NumPy)}')
print(f'Our R:\n{str(R_custom)}')
print(f'NumPy R:\n{str(R_NumPy)}')

Our Q:
[[ 0.70710679  0.40824828 -0.5773503 ]
 [ 0.70710679 -0.40824828  0.5773503 ]
 [ 0.          0.81649655  0.57735025]]
NumPy Q:
[[-0.70710678  0.40824829 -0.57735027]
 [-0.70710678 -0.40824829  0.57735027]
 [-0.          0.81649658  0.57735027]]
Our R:
[[1.4142135  0.70710677 0.70710677]
 [0.         1.2247449  0.4082483 ]
 [0.         0.         1.1547005 ]]
NumPy R:
[[-1.41421356 -0.70710678 -0.70710678]
 [ 0.          1.22474487  0.40824829]
 [ 0.          0.          1.15470054]]


However, both algorithms obtain the same matrix when Q and R are multiplied.

In [6]:
print('Our Algorithm:\n', np.round(Q_custom @ R_custom))
print('NumPy Algorithm:\n', np.round(Q_NumPy @ R_NumPy))

Our Algorithm:
 [[ 1.  1. -0.]
 [ 1. -0.  1.]
 [ 0.  1.  1.]]
NumPy Algorithm:
 [[ 1.  1. -0.]
 [ 1. -0.  1.]
 [ 0.  1.  1.]]


In [7]:
# Singular Matrix Example

S = np.array([
    [1, -1, 0],
    [1,  0, 1],
    [0,  1, 1]
])

det = np.linalg.det(S)
print(f'Non singular ({det})' if det != 0 else 'Singular')

Singular


In [9]:
# factorization of a singular matrix

Qs, Rs = factorizer(S)  # our algorithm
print(f'Q=\n{Qs}')
print(f'R=\n{Rs}')

print('\nIf we multiply the matrices, we get false results...')
print(f'S = Q x R =\n{Qs @ Rs}')

print(f'===========================================================================')

Qs, Rs = np.linalg.qr(S)  # NumPy's algorithm
print(f'Q=\n{Qs}')
print(f'R=\n{Rs}')

print('\nWhere as with NumPy, it can still decompose the matrix S and obtain correct results upon multiplication...')
print(f'S = Q x R =\n{np.round(Qs @ Rs)}')

Q=
[[ 0.70710679 -0.40824828         nan]
 [ 0.70710679  0.40824828         nan]
 [ 0.          0.81649655         nan]]
R=
[[ 1.4142135  -0.70710677  0.70710677]
 [ 0.          1.2247449   1.2247449 ]
 [ 0.          0.          0.        ]]

If we multiply the matrices, we get false results...
S = Q x R =
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
Q=
[[-0.70710678  0.40824829  0.57735027]
 [-0.70710678 -0.40824829 -0.57735027]
 [-0.         -0.81649658  0.57735027]]
R=
[[-1.41421356e+00  7.07106781e-01 -7.07106781e-01]
 [ 0.00000000e+00 -1.22474487e+00 -1.22474487e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.52468327e-16]]

Where as with NumPy, it can still decompose the matrix S and obtain correct results upon multiplication...
S = Q x R =
[[ 1. -1. -0.]
 [ 1.  0.  1.]
 [ 0.  1.  1.]]
